# Emergency Response Ops – Portfolio
Raw → Cleaned → Insights (Severity Mix, Agent Performance, Hotspots)

**Goal:** Standardize severity values, compute resolution time, and surface 3 business insights.


In [ ]:
USE WAREHOUSE SYSTEM$STREAMLIT_NOTEBOOK_WH;
USE DATABASE emergency_response;
USE SCHEMA public;


 # Quick peek at the data 

### Cases table

In [ ]:
SELECT * FROM cases LIMIT 5;

### Agents table

In [ ]:
SELECT * FROM agents LIMIT 5;

### Travelers table 

In [ ]:
SELECT * FROM travelers LIMIT 5;

# Clean the data

In [ ]:
CREATE OR REPLACE VIEW cases_clean AS
SELECT
  case_number,
  prod_id,
  traveler_id,
  subject,
  case_owner_alias,
  UPPER(TRIM(severity)) AS severity_cleaned,
  country,
  city,
  agent_id,
  date_time_opened AS dt_opened,
  date_time_closed AS dt_closed,
  
  -- compute duration in minutes if both timestamps are present
  IFF(
    date_time_opened IS NULL OR date_time_closed IS NULL,
    NULL,
    DATEDIFF(minute, date_time_opened, date_time_closed)
  ) AS resolution_minutes
FROM cases;


### Quick sanity check 

In [ ]:
--Peek 
SELECT * FROM cases_clean LIMIT 10;

In [ ]:
-- How many rows have no resolution time yet?
SELECT COUNT(*) AS rows_with_null_duration
FROM cases_clean
WHERE resolution_minutes IS NULL;

In [ ]:
-- Severity distribution (after cleaning)
SELECT severity_cleaned, COUNT(*) AS total_cases
FROM cases_clean
GROUP BY severity_cleaned
ORDER BY total_cases DESC;

# Insight 1-Case Volume & Severity Trends 
### Which severities are most common? 
**Why it matters:** shows the mix of work by urgency; informs staffing & triage focus.

In [ ]:
SELECT
  severity_cleaned,
  COUNT(*) AS total_cases
FROM cases_clean
GROUP BY severity_cleaned
ORDER BY total_cases DESC;


In [ ]:
import altair as alt
import streamlit as st

# Get results from SQL cell 13
df = cell13.to_pandas()

# Sort descending
df = df.sort_values("TOTAL_CASES", ascending=False)

# Base bar chart
chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("SEVERITY_CLEANED:N",
                title="Severity",
                sort="-y",
                axis=alt.Axis(labelAngle=0)),   
        y=alt.Y("TOTAL_CASES:Q", title="Total Cases"),
        color=alt.Color("SEVERITY_CLEANED:N", legend=None),   
        tooltip=["SEVERITY_CLEANED", "TOTAL_CASES"]
    )
)

# Add text labels above bars
text = chart.mark_text(
    align="center",
    baseline="bottom",
    dy=-2,
    color="black"
).encode(
    text="TOTAL_CASES:Q"
)

# Combine chart + text
final_chart = chart + text

st.subheader("Cases by Severity")
st.altair_chart(final_chart, use_container_width=True)


### Summary — Case Severity Mix
- Most cases are **Severity D (low urgency)**, followed by **Severity C**.  
- High-severity cases (A/B) are rare, making up less than 10% of the total volume.  
- This indicates that the bulk of operational workload is **routine, lower-priority support**.  

**Action:**  
Optimize staffing for **high-volume, low-severity triage**, while maintaining a **specialized escalation path** for Severity A/B cases to ensure critical issues are resolved quickly without over-allocating resources.  


# Insight 2-Agent Performance
### Which agents close cases fastest? Useful for coaching and scheduling. 

In [ ]:
SELECT
  a.agent_id,
  a.shift_type,
  a.shift_region,
  COUNT(*) AS cases_handled,
  ROUND(AVG(resolution_minutes), 2) AS avg_resolution_minutes
FROM cases_clean c
JOIN agents a
  ON c.agent_id = a.agent_id
WHERE resolution_minutes IS NOT NULL
GROUP BY a.agent_id, a.shift_type, a.shift_region
ORDER BY avg_resolution_minutes ASC;


In [ ]:
-- Fastest 10 agents
SELECT *
FROM (
  SELECT a.agent_id, a.shift_type, a.shift_region,
         COUNT(*) AS cases_handled,
         ROUND(AVG(resolution_minutes), 2) AS avg_resolution_minutes
  FROM cases_clean c
  JOIN agents a ON c.agent_id = a.agent_id
  WHERE resolution_minutes IS NOT NULL
  GROUP BY a.agent_id, a.shift_type, a.shift_region
)
ORDER BY avg_resolution_minutes ASC
LIMIT 10;

In [ ]:
import altair as alt
import streamlit as st

# 1) Get the SQL results into pandas
#   Replace top10agents with your SQL cell name (or use cell15 / cell16 etc.)
df = ten_fastest_agents.to_pandas()

# 2) Build the chart (horizontal; fastest on top)
chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        y=alt.Y("AGENT_ID:N", title="Agent ID", sort="x"),     # fastest (smallest x) at top
        x=alt.X("AVG_RESOLUTION_MINUTES:Q", title="Avg Resolution (mins)"),
        color=alt.Color("AGENT_ID:N", legend=None),             # different color per bar
        tooltip=["AGENT_ID", "SHIFT_TYPE", "SHIFT_REGION", "CASES_HANDLED", "AVG_RESOLUTION_MINUTES"]
    )
)

# 3) Add numbers to bars
text = chart.mark_text(align="left", baseline="middle", dx=4, color="black").encode(
    text=alt.Text("AVG_RESOLUTION_MINUTES:Q", format=".1f")
)

# 4) RENDER the chart (this line is what was missing)
st.subheader("Top 10 Fastest Agents (Avg Resolution Minutes)")
st.altair_chart(chart + text, use_container_width=True)


### Summary — Top 10 Fastest Agents
- Fastest agents close cases in **~160–190 minutes** on average; several sit well **under 200 minutes**.
- The leaderboard highlights consistent, repeatable performance — not one-offs — and shows **clear separation** from slower peers (>200 min).
- **Action:** Use these agents’ workflows for peer coaching; align complex queues to proven fast performers, and review slower agents’ cases for bottlenecks (handoffs, escalations, region/time coverage).


# Insight 3- Geographic Hotspots 
### Where do issues occur most often? This highlights countries where travelers face the most disruptions. 

In [ ]:
SELECT
  country,
  COUNT(*) AS case_count,
  ROUND(AVG(resolution_minutes), 2) AS avg_resolution_minutes
FROM cases_clean
WHERE country IS NOT NULL AND country <> ''
GROUP BY country
ORDER BY case_count DESC
LIMIT 10;


In [ ]:
import altair as alt
import streamlit as st

# Pull SQL results into pandas (replace with your SQL cell number)
df = cell17.to_pandas()   # update to your actual cell number

# Sort by case_count descending
df = df.sort_values("CASE_COUNT", ascending=False)

# Build chart
chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("COUNTRY:N", title="Country", sort="-y",
                axis=alt.Axis(labelAngle=-30)),   
        y=alt.Y("CASE_COUNT:Q", title="Case Count"),
        color=alt.Color("COUNTRY:N", legend=None),   
        tooltip=["COUNTRY", "CASE_COUNT", "AVG_RESOLUTION_MINUTES"]
    )
)

# Add numbers above bars
text = chart.mark_text(
    align="center",
    baseline="bottom",
    dy=-2,
    color="black"
).encode(
    text="CASE_COUNT:Q"
)

# Combine bar + labels
final_chart = chart + text

st.subheader("Cases by Country (Top 10)")
st.altair_chart(final_chart, use_container_width=True)


### Summary — Geographic Hotspots
- The highest number of cases come from **Italy, United States, and Japan**, each with ~190–200 cases.  
- These hotspots indicate regions where travelers face more frequent disruptions, likely influenced by local infrastructure, demand patterns, or travel volumes.  
- Case volumes remain consistently high across multiple countries, reinforcing the need for global resource planning.  

**Action:**  
Prioritize **regional support strategies** in top-case countries (Italy, U.S., Japan) by increasing local partnerships, dedicating more regional agents, and providing **proactive traveler communication**. This can reduce case escalations and improve overall service delivery.  


# ✅ Final Wrap-Up & Recommendations  

This analysis of Emergency Response case data highlights key patterns in **severity distribution, agent performance, and geographic hotspots**.  

### Key Insights  
1. **Case Severity Mix**  
   - Majority of cases are **low urgency (Severity D & C)**.  
   - High-severity cases (A/B) are rare (<10%).  
   👉 Resources should be optimized for **high-volume, low-severity triage**, with clear escalation for critical cases.  

2. **Agent Performance**  
   - Top-performing agents resolve cases in **~160–190 minutes** on average.  
   - A few agents stand out as both **fast and handling high volume**, while others take significantly longer.  
   👉 Recognition and **peer coaching** from top performers can uplift slower agents. Shift planning can route complex or urgent cases to proven fast agents.  

3. **Geographic Hotspots**  
   - Highest case volumes occur in **Italy, U.S., and Japan** (~190–200 each).  
   - These regions show consistent traveler disruption.  
   👉 Establish **regional partnerships**, expand local support coverage, and provide proactive communication to travelers in these markets.  

---

### 📌 Recommendations  
- **Resource Allocation:** Staff more heavily for **routine triage** while protecting bandwidth for rare but critical Severity A/B cases.  
- **Agent Development:** Use performance metrics to drive **training, recognition, and scheduling strategies**. Highlight “star agents” as process role models.  
- **Regional Strat**
